In [1]:
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys
import pandas as pd
import numpy as np
import sys
from collections import OrderedDict
import json
import pysolr
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import urllib
import json
import math
import pickle
from joblib import dump, load
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [3]:
def generate_note_txt_solr_query(input_ids):
    
    list_of_ids = [str(x) for x in input_ids['report_id']]
    
    ids = '"{}"\\OR\\"'.format(list_of_ids[0])
    ids += '"\\OR\\"'.join(list_of_ids[1:])
    
    q = "http://18.220.133.76:8983/solr/sample/tvrh?q=report_id:{}&start=0&rows=10000&fl=id,report_text&wt=json".format(ids)
    return q
    

In [4]:
def generate_notes_df(sample_ids):

    notes_df = pd.DataFrame(np.zeros([sample_ids.shape[0], 2]), columns=['report_id', 'text'])
    solr_query = generate_note_txt_solr_query(sample_ids)

    connection = urllib.request.urlopen("{}".format(solr_query))
    response = json.load(connection)
    
    notes_df.loc[:, 'report_id'] = sample_ids['report_id']
 
    for i in range(len(sample_ids['report_id'])):
        notes_df.loc[i, 'text'] = response['response']['docs'][i]['report_text']


    return notes_df

In [44]:
def predict_primitive_label(path_to_vectorizer, path_to_models_dir, primitive, new_note_text):

    with open(path_to_vectorizer, 'rb') as f:
        vectorizer = pickle.load(f)
        
        new_x =  vectorizer.transform([new_note_text])

    with open("{}/{}_trained_dnn_smotev2.joblib".format(path_to_models_dir, primitive), 'rb') as m:
        
        clf =load(m)
        
        y_pred = clf.predict(new_x)
        print(y_pred)   
    

In [47]:
predict_primitive_label("./trained_vectorizer.pk","./models", "is_female", "the patient has hypertension")

[0]
